# Contribute

## Wanted

- More test coverage
- More documentation
- More syntax cases
- Useful magics
- JupyterLab MIME renderer for Robot `output.xml`

## Hack

The Kernel and the JupyterLab extension are developed in tandem in the same environment.

[Fork the repo](https://github.com/gtri/irobotframework), and clone it locally...

    git clone https://github.com/gtri/irobotframework

Get [Miniconda](https://conda.io/miniconda.html).

Get [anaconda-project](https://github.com/Anaconda-Platform/anaconda-project)...

    conda install -n base anaconda-project

In the repo, run **bootstrap** the `default` environment...

    anaconda-project run bootstrap
    anaconda-project run jlpm bootstrap

Start the **TypeScript** watcher...

    anaconda-project run jlpm watch

...and in a separate terminal, start the **JupyterLab** watcher...

    anaconda-project run lab --watch

Start **doing work**, and enjoy live rebuilding, available after refreshing your page. Kernel changes will require restarting your kernel.

Keep the [quality](#Quality) high by running (and adding) tests, and ensuring your code meets the formatting standards.

Make a [pull request](https://github.com/gtri/irobotframework)!

## Quality

Code quality is assured in a separate `test` environment. Get it ready with **bootstrap**:

    anaconda-project run --env-spec test bootstrap
    anaconda-project run --env-spec test jlpm bootstrap

Who wants ugly code? Squash all your syntax opinions...

    anaconda-project run test:lint

Try to keep the coverage up by adding [unit tests](https://github.com/gtri/irobotframework/src/irobotframework/tests)...

    anaconda-project run test:py

Make sure everything is on the up-and-up with [integration tests](https://github.com/gtri/irobotframework/src/atest)...

    anaconda-project run test:robot

### Syntax Highlighting

The `jupyterlab-robotframework` syntax highlighting mode is tested against known-good fixtures from:
- the Robot Framework User's Guide (RFUG)
- some degenerate examples

The `atest/fixtures/highlighting` has two parallel directories, `samples` and `tokens`. Under test, each `sample` of Robot syntax is added to a notebook, and the stream of tokens compared with the expected `tokens`. For example,

```robotframework
*** Test Cases ***
A Test Case
  Log  Hello World
```

expects the following `.tokens` file:

```
header link
header string
keyword
string
string
```

During testing, the observed tokens are also written out to the `_testoutput`. After a test run, `atest/tools/SyntaxChecker.ipynb` can be used to visually compare non-agreeing observations, and copy them to `atext/fixtures` if correct:

    anaconda-project run --env-spec hack bootstrap
    anaconda-project run --env-spec hack lab

#### Rebuilding the Syntax Samples

The RFUG samples can be rebuilt from the `robotframework` source (included as a submodule in `vendor`).

    git submodule update --init
    anaconda-project run build:rfug
    anaconda-proejct run test:robot

When a new version of Robot Framework is released, new values might appear and old ones might disappear, as the actual content of the robot code blocks from the generated HTML is hashed. Since these changes may include typo fixes, it's _mostly harmless_ to remove stale hashes and their corresponding tokens.

#### Syntax: Future Work
- Because they are already _in_ the submodule, the 1000+ `.robot` files in the Robot test suite itself would be great to use as a larger baseline for visual comparison: 
  - It would only take a few hours initially, and less per subsequent release, to validate all of the positive cases... though syntax bugs may well be discovered, which could take more time
- Going beyond known fixtures, a [hypothesis](https://hypothesis.readthedocs.io/en/latest/)-based generator of syntactically possible documents would be useful

## Document
Documentation is pretty important, as is pretty documentation. Docs are built from `irobotframework` and IPython notebooks with [nbsphinx](https://github.com/spatialaudio/nbsphinx), and can be updated automatically on save with:

    anaconda-project run --env-spec docs bootstrap
    anaconda-project run build:docs:watch

> This will tell you which port to visit in your browser.

This contributes to [Quality](#Quality), as the notebooks are executed prior to run, though some of them _anticipate_ failure, and proceed anyway.

> Don't check in notebooks with outputs!

## Build

This is mostly handled by CI, but you can **build** all of the output packages into `./dist`.

Clear out your repo (or do a fresh checkout)...
   
    git clean -dxf  # DANGER this will delete everything not checked into git!
    anaconda-project run build:pip:sdist
    anaconda-project run build:pip:wheel
    anaconda-project run build:npm
    anaconda-project run build:conda
    anaconda-project run build:docs


## Release
This is _also_ mostly handled by CI, but here is the rough outline.

Preflight the release...

    anaconda-project run test:release
   
If you had to make any changes, go ahead and fix them, and rebuild everything.

    anaconda-project run jlpm lerna publish
    twine upload dist/pip/*

Wait for the bot to pick up the changes on `conda-forge`.